In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import scipy
import math
import os
import sys
print(f'python version: {sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'nibabel version: {nib.__version__}')


NEAR_ZERO_THRESHOLD = 1e-6
def int2seg(intensity):
    if intensity <= 7:
        return f"C{intensity}"
    elif intensity >7 and intensity <= 19:
        return f"T{intensity-7}"
    else:
        return f"L{intensity-19}"
    
df = pd.read_csv("mendez_df.csv")
df.head()

python version: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:06) 
[Clang 14.0.6 ]
numpy version: 1.23.5
pandas version: 1.5.3
nibabel version: 5.1.0


,Segment,MEAN,STD,Mesure
0,C2,11.3,1.5,DREZ length
1,C2,12.6,0.9,Seg.L at dorsal CE
2,C2,13.1,1.0,Seg.L at BE
3,C2,24.4,0.8,Inf.Art facet to CR distance
4,C2,15.5,4.2,IF to RR distance


In [2]:
sct_dir = '../Code/spinalcordtoolbox/data/PAM50/template/'
print(sct_dir)

../Code/spinalcordtoolbox/data/PAM50/template/


In [3]:
# Import foramen data
#foramen_level = nib.load('PAM50_intervertebral_foramen_sandrine.nii.gz') # TODO add path
foramen_level = nib.load('test_foramen.nii.gz')

x_f,y_f,z_f = np.where(foramen_level.get_fdata()>NEAR_ZERO_THRESHOLD)
print(x_f, z_f)

[37 40 41 41 42 42 42 42] [932 731 691 898 765 797 830 865]


In [4]:
# Import Centerline data
center = nib.load(os.path.join(sct_dir, 'PAM50_centerline.nii.gz'))
x_c, y_c, z_c = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)
coord_ctl = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)


# Import Cord data
PATH = os.path.join(sct_dir, 'PAM50_cord.nii.gz')
cord = nib.load(PATH)
p50_mask = cord.get_fdata()
x_cord, y_cord, z_cord = np.where(p50_mask>0)

## Estimate without STD

## Estimate with STD 

In [58]:
# Take smallest distance bewtenn SC centerline (shifted by average diameter at dorsal rootlet entry and intervertebral foramen) (Sandrine)

z_ref =  np.array(range(min(z_c), max(z_c) + 1))
z_c = z_ref
for i in range(len(x_f)):
    # Get coordinates of intervertebral foramen
    x = x_f[i]
    y = y_f[i]
    z = z_f[i]
    # Get level
    level = int(foramen_level.get_fdata()[x,y,z])
    print('Level', level)
    # Get correspondance of level number (2 --> C2, etc.)
    seg = int2seg(level)
    print(seg)
    # Get mendez measures for the corresponding level
    seg_df = df[df.Segment == seg]
    RR = seg_df[seg_df.Mesure == "IF to RR distance"]
    CR = seg_df[seg_df.Mesure == "IF to CR distance"]
    W = seg_df.loc[seg_df.Mesure == "Dorsal width"].iloc[0]
    w_mean = W.MEAN # get mean value of dorsal width
    rr_mean = RR.MEAN.to_list()[0] # Get mean distance from rostral rootlet to interverteral foramen
    cr_mean = CR.MEAN.to_list()[0] # Get mean distance from caudal rootlet to interverteral foramen
    rr_std = RR.STD.to_list()[0] # Get std for mean distance from rostral rootlet to interverteral foramen
    cr_std = CR.STD.to_list()[0] # Get std for mean distance from caudal rootlet to interverteral foramen
    # Get distance for +/- 95% distribution of distance
    min_rr = rr_mean - 2*rr_std
    max_rr = rr_mean + 2*rr_std
    min_cr = cr_mean - 2*cr_std
    max_cr = cr_mean + 2*cr_std
    print("Rostral dist", max_rr, rr_mean , min_rr)
    print("Caudal dist", max_cr, cr_mean , min_cr)
    # Compute the 3D euclidean distance between intervertebral foramen and the back of the SC at dorsal width/2 lateral offset ( in mm)
    new_x = (x_c - w_mean)[0] # x value centerline - dorsal width /2 offset
    new_z = min(np.where(cord.get_fdata()[int(new_x),:,z] >0)[0]) # y value of the SC border at x = new_x and z = foramen
    distance_foramen_ctl = np.sqrt((x_c - w_mean - x)**2 + (new_z - y)**2  + (z_c - z)**2)*0.5  # 0.5 --> pix dim of PAM50 TODO: remove hardcode
    # List with estimation of all z value 
    all_level = []
    for distance in [max_rr,min_rr,rr_mean,max_cr, min_cr, cr_mean]:
        # Get the closest distance to the rostral rootlet compared to the Mendez value
        rostral_diff = np.array([np.abs(i - distance) for i in distance_foramen_ctl])
        # Only use slices higher than the foramen
        estimate = np.argmin(rostral_diff[-len(z_c[z_c>z])-1::])
        z_ref_i = z_ref[-len(z_c[z_c>z])-1::]
        # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
        value = z_ref_i[estimate]
        all_level.append(int(value))

    print('Rostral', rr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][rostral], all_level[2] , z)
    print('Caudal', cr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][caudal],all_level[5] , z)
    # Set slices
    cord_mask = np.copy(p50_mask)
    # Max to mean rostral
    for rng in range(all_level[2], all_level[0]):
        if abs(all_level[0]-all_level[2]) !=0:
            proba = 19*abs(rng-all_level[2])/(abs(all_level[0]-all_level[2])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = 20-proba
    # Mean to min rostral 
    for rng in range(all_level[1], all_level[2]):
        if abs(all_level[2]-all_level[1]) != 0:
            proba = 19*abs(rng-all_level[1])/(abs(all_level[2]-all_level[1])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = proba
    # Max to mean caudal
    for rng in range(all_level[5], all_level[3]):
        if abs(all_level[3]-all_level[5]) !=0:
            proba = 19*abs(rng-all_level[5])/(abs(all_level[3]-all_level[5])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = 20-proba
    # Mean to min caudal
    for rng in range(all_level[4], all_level[5]):
        if abs(all_level[5]-all_level[4]) != 0:
            proba = 19*abs(rng-all_level[4])/(abs(all_level[5]-all_level[4])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = proba
    # Mean value caudal and rostral = 20
    for l in (all_level[2], all_level[5]):
        cord_mask[:, :, l][cord_mask[:, :, l]>0] = 20
    cord_mask[:, :, :][cord_mask[:, :, :]==1] = 0
    test_img = nib.Nifti1Image(cord_mask, header=cord.header, affine=cord.affine)
    nib.save(test_img, f'Outputs/TEST/level_{level}.nii.gz')  # TO CHANGE path output

Level 2
C2
Rostral dist 7.1 15.5 23.9
Caudal dist 9.9 10.6 11.299999999999999
Rostral 15.5 12.913310961949302 949 932
Caudal 10.6 12.913310961949302 932 932
[972, 932, 949, 932, 932, 932]
Level 8
T1
Rostral dist 20.3 28.1 35.9
Caudal dist 14.1 17.0 19.9
Rostral 28.1 15.241062955056643 778 731
Caudal 17.0 15.241062955056643 746 731
[796, 758, 778, 765, 731, 746]
Level 9
T2
Rostral dist 22.199999999999996 33.4 44.6
Caudal dist 18.2 22.0 25.8
Rostral 33.4 14.206133358518072 751 691
Caudal 22.0 14.206133358518072 725 691
[776, 725, 751, 743, 696, 725]
Level 3
C3
Rostral dist 11.299999999999997 16.299999999999997 21.299999999999997
Caudal dist 8.100000000000001 10.8 13.5
Rostral 16.299999999999997 15.851753846183708 906 898
Caudal 10.8 15.851753846183708 898 898
[926, 898, 906, 905, 898, 898]
Level 7
C7
Rostral dist 18.5 20.1 21.700000000000003
Caudal dist 10.9 11.8 12.700000000000001
Rostral 20.1 15.019906790656194 792 765
Caudal 11.8 15.019906790656194 765 765
[796, 787, 792, 765, 765, 76